Walk Through for Knowledge Graph Testing

In [ ]:
from multivac.src.rdf_graph.map_queries import *
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
args_dict = {'dir': 'data',
             'out': 'models',
             'glove': 'models/glove.42B.300d',
             'run': 'model',
             'model': 'transe',
             'threshold': 0.1,
             'num_top_rel': 10}

args_dict['search'] = '/Users/ben_ryan/Documents/DARPA_ASKE/Phase_II/openke_experiments/query_tester.csv'


In [ ]:
con = config.Config()
con.set_in_path(args_dict['dir']+os.path.sep)
con.set_work_threads(8)
con.set_dimension(100)
con.set_test_link_prediction(True)
con.set_test_triple_classification(True)

files = glob.glob(os.path.join(args_dict['out'],'*tf*'))
times = list(set([file.split('.')[2] for file in files]))
ifile = max([datetime.strptime(x, '%d%b%Y-%H:%M:%S') for x in times]).strftime('%d%b%Y-%H:%M:%S')
con.set_import_files(os.path.join(args_dict['out'], 'model.vec.{}.tf'.format(ifile)))

con.init()
kem = set_model_choice(args_dict['model'])
con.set_model(kem)

In [ ]:
annots =  "tokenize ssplit pos depparse natlog openie ner coref",
props  = {"openie.triple.strict": "true",
          "openie.openie.resolve_coref": "true"}

parser = StanfordParser(annots=annots, props=props)

# glove = loadGloveModel(args_dict['glove'], verbose)
glove_vocab, glove_emb = load_word_vectors(args_dict['glove'])


In [ ]:
files = glob.glob(os.path.join(con.in_path,'*.txt'))
rel_file = max([file for file in files if re.search('relation', file)],
               key=os.path.getctime)
ent_file = max([file for file in files if re.search('entity', file)],
               key=os.path.getctime)

entities = pd.read_csv(ent_file, sep='\t', 
                       names=["Ent","Id"], skiprows=1)
relations = pd.read_csv(rel_file, sep='\t', 
                        names=["Rel","Id"], skiprows=1)


In [ ]:
queries = pd.read_csv(args_dict['search'])
parse = lambda z: stanford_parse(parser, z, sub_rdfs=True).get_rdfs(use_tokens=False, 
                                                                    how='longest')
triples = queries.Query.apply(parse)
results = triples.apply(lambda x: get_answers(con, x, 
                                              glove_vocab, 
                                              glove_emb, 
                                              entities, 
                                              relations,
                                              args_dict['num_top_rel'], 
                                              args_dict['threshold']))

In [ ]:
for result in results:
    print(result)